In [1]:
# !nvidia-smi

In [2]:
# !pip install transformers

In [ ]:
# !pip uninstall manim

^C


In [ ]:
# !pip install numpy==1.26.4

# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [5]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [6]:
# !pip install --upgrade datasets fsspec

In [7]:
# !pip install evaluate


# Libraries

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset
from evaluate import load

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt_tab")


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Reek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


c:\Users\Reek\anaconda3\envs\LLMs\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset_samsum = load_dataset("knkarthick/dialogsum")

In [9]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [10]:
dataset_samsum['train']['dialogue'][1]

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [11]:
dataset_samsum['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

In [12]:
dataset_samsum['train'][1]['topic']

'vaccines'

In [13]:
dataset_samsum['train'][1]

{'id': 'train_1',
 'dialogue': "#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.",
 'summary': 'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.',
 'topic': 'vaccines'}

In [14]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")

Split lengths: [12460, 500, 1500]
Features: ['id', 'dialogue', 'summary', 'topic']


In [15]:
print("\nDialogue:")
print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")
print(dataset_samsum["test"][1]["summary"])

print("\Topic:")
print(dataset_samsum["test"][1]["topic"])


Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with t

In [16]:
def convert_examples_to_features(example_batch):
    # Prepend topic to dialogue input
    input_text = ["Topic: " + t + " Dialogue: " + d for t, d in zip(example_batch["topic"], example_batch["dialogue"])]
    
    input_encodings = tokenizer(input_text, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [17]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

c:\Users\Reek\anaconda3\envs\LLMs\lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [18]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12460
})

In [19]:
dataset_samsum_pt['train']['input_ids'][1]

[21916,
 151,
 16595,
 30275,
 151,
 1768,
 33892,
 740,
 4009,
 151,
 8087,
 5390,
 107,
 9235,
 108,
 199,
 133,
 119,
 174,
 152,
 1768,
 33892,
 522,
 4009,
 151,
 8087,
 982,
 107,
 22579,
 107,
 1205,
 1226,
 2041,
 119,
 107,
 26091,
 111,
 125,
 127,
 264,
 118,
 169,
 16595,
 107,
 1768,
 33892,
 740,
 4009,
 151,
 3695,
 210,
 107,
 1593,
 131,
 116,
 236,
 108,
 992,
 112,
 169,
 19138,
 1093,
 108,
 26091,
 148,
 915,
 169,
 30047,
 554,
 108,
 41586,
 34068,
 116,
 111,
 48376,
 596,
 3901,
 107,
 285,
 117,
 1265,
 590,
 459,
 108,
 167,
 178,
 117,
 640,
 118,
 48376,
 202,
 108,
 8097,
 49042,
 111,
 2587,
 1483,
 6366,
 3901,
 107,
 1768,
 33892,
 522,
 4009,
 151,
 463,
 160,
 24387,
 7655,
 111,
 14663,
 9020,
 152,
 1768,
 33892,
 740,
 4009,
 151,
 1894,
 108,
 125,
 137,
 209,
 361,
 342,
 219,
 118,
 239,
 108,
 111,
 244,
 114,
 932,
 113,
 899,
 125,
 137,
 15079,
 109,
 1004,
 107,
 1768,
 33892,
 522,
 4009,
 151,
 4810,
 108,
 255,
 107,
 7167,
 108,
 125,
 

In [20]:
dataset_samsum_pt['train']['attention_mask'][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [21]:
dataset_samsum_pt['train']['labels'][1]

[5390,
 9235,
 839,
 26091,
 118,
 169,
 16595,
 107,
 982,
 107,
 22579,
 5723,
 109,
 1093,
 111,
 237,
 1106,
 26091,
 114,
 10733,
 107,
 1]

In [22]:
# Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)



In [23]:
# !pip uninstall transformers -y
# !pip install transformers==4.40.0


In [24]:
import transformers
print(transformers.__version__)


4.40.0


In [25]:

from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)


In [26]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])



In [27]:

trainer.train()

  0%|          | 0/93 [00:00<?, ?it/s]

{'loss': 3.2639, 'grad_norm': 20.45037841796875, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.11}
{'loss': 3.1594, 'grad_norm': 14.561914443969727, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.21}
{'loss': 3.0684, 'grad_norm': 11.114509582519531, 'learning_rate': 3e-06, 'epoch': 0.32}
{'loss': 2.8204, 'grad_norm': 14.0777587890625, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.43}
{'loss': 2.6146, 'grad_norm': 16.843915939331055, 'learning_rate': 5e-06, 'epoch': 0.53}
{'loss': 2.7313, 'grad_norm': 21.220874786376953, 'learning_rate': 6e-06, 'epoch': 0.64}
{'loss': 2.3978, 'grad_norm': 17.45792579650879, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.75}
{'loss': 2.3615, 'grad_norm': 35.11902618408203, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.85}
{'loss': 2.3038, 'grad_norm': 7.875046253204346, 'learning_rate': 9e-06, 'epoch': 0.96}
{'train_runtime': 2741.1847, 'train_samples_per_second': 0.547, 'train_steps_per_second': 0.034, 'train_loss': 2.7293900520

TrainOutput(global_step=93, training_loss=2.7293900520570817, metrics={'train_runtime': 2741.1847, 'train_samples_per_second': 0.547, 'train_steps_per_second': 0.034, 'total_flos': 909278332846080.0, 'train_loss': 2.7293900520570817, 'epoch': 0.992})

In [28]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that we can process simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


In [35]:
dataset_samsum['test'][0]

{'id': 'test_0_1',
 'dialogue': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much tim

In [36]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="dialogue",
                                column_summary="summary"):
    
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=128
        )

        # Finally, we decode the generated texts,
    # replace the <pad> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
                            for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score




In [37]:
import evaluate
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load('rouge')


In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum["test"][:10],  # or full dataset_samsum["test"]
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=8,
    device='cuda'  # or 'cpu'
)



100%|██████████| 2/2 [04:52<00:00, 146.09s/it]


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [40]:
rouge_dict = {rn: round(score[rn], 4) for rn in rouge_names}


pd.DataFrame(rouge_dict, index=[f'pegasus'])


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.0051,0.0,0.0051,0.0049


In [46]:
model_pegasus.save_pretrained("pegasus_dialogsum")
tokenizer.save_pretrained("tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

# for custom tokenizer

In [48]:
# Load
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

# Prediction
gen_kwargs = {
    "length_penalty": 0.8, # close to 1 long output close to 0 short output
    "num_beams": 8,
    "max_length": 128
}

# Sample input and reference from SAMSum
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

# Create summarization pipeline
from transformers import pipeline

# pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)
pipe = pipeline("summarization", model="./pegasus_dialogsum", tokenizer=tokenizer)

# pipe = pipeline("summarization", model="google/pegasus-samsum", tokenizer="google/pegasus-samsum")


# Display input dialogue
print("Dialogue:")
print(sample_text)

# Generate summary
generated_summary = pipe(sample_text)[0]['summary_text']

# Display result
print("\nGenerated Summary:")
print(generated_summary)

print("\nReference Summary:")
print(reference)



Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with th

In [49]:
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])



Model Summary:
All office communications are restricted to email correspondence and official memos.<n>The use of Instant Message programs by employees during working hours is strictly prohibited.<n>Employees who persist in using Instant Messaging will first receive a warning and be placed on probation. At second offense, the employee will face termination.
